# Derived from AUTOMATIC1111's Stable Diffusion WebUI 

https://github.com/AUTOMATIC1111/stable-diffusion-webui



# RUN ALL CELLS BELOW AFTER MACHINE BOOT



If you need to soft-restart the webui, just stop and start the final cell

# FIRST CELL SETS UP THE ENVIRONMENT



In [1]:
# Choose where to store your model checkpoints.

# Free Tier
# model_storage_dir = '/tmp/stable-diffusion-models'

# Paid Tier
model_storage_dir = '/storage/models'



# Optional path settings
repo_storage_dir = '/storage/stable-diffusion'         # Where to store your Stable Diffusion-related files.

export_storage_dir = '/notebooks/exports'              # Where the generated images will be exported to.

pip_cache_dir = None                                   # The installer can cache pip wheels so you don't have to re-download them
                                                       # every time you start the machine. I recommed setting it to '/storage/pip/cache'


# Other optional settings
# You don't have to change these if you don't want to.

symlink_to_notebooks = True                            # Enables the creation of symlinks back to /notebooks/

activate_xformers = True                               # Enables the xformers optimizations using pre-built wheels.
                                                       # Setting to True will automatically set up your environment/machine for xformers. 

link_novelai_anime_vae = True                          # Enables the linking of animevae.pt to each of the NovelAI models.
                                                       # Set to True if you've downloaded both the NovelAI models and hypernetworks.

activate_deepdanbooru = False                          # Enable and install DeepDanbooru -> https://github.com/KichangKim/DeepDanbooru

activate_medvram = False                                # Enable medvram option.
                                                       # These are model optimizations which will reduce VRAM usage at the expense of some speed.
                                                       # Set to False if you have a lot of VRAM.

disable_pickle_check = False                           # Disable the automatic check for unexpected data in model files.
                                                       # Leave this set to False unless you have a reason to disable the check.

gradio_port = False                                    # Launch Gradio on a specific port. Set to False to let Gradio choose a port.
                                                       # This disables online Gradio app mode and you will only be able to access it on your local network.

gradio_auth = False                                    # Enable gradio_auth and insecure-extension-access option.
                                                       # Set to a username:password (for example: "me:password") to enable.

search_paperspace_datasets = True                      # Enable searching for checkpoints in /datasets to link to the webui

ui_theme = None                                        # Set the WEB UI theme. Values can be None (default) or 'dark'.

insecure_extension_access = False                      # Force enable extensions without a password.
                                                       # If you don't set a password anyone can install and run arbitrary code on your machine!
                                                       # Instead, use gradio_auth which will automatically enable extensions when set.

gradio_queue = False                                   # Uses gradio queue; experimental option; breaks restart UI button.

install_pip_xformers = False                           # Install xformers through pip. Probably won't work because it needs Torch 2.0

# ===================================================================================================
# Save variables to Jupiter's temp storage so we can access it even if the kernel restarts.
%store symlink_to_notebooks model_storage_dir repo_storage_dir export_storage_dir activate_xformers link_novelai_anime_vae activate_deepdanbooru activate_medvram disable_pickle_check gradio_port gradio_auth search_paperspace_datasets ui_theme insecure_extension_access pip_cache_dir gradio_queue install_pip_xformers

try:
    %store -r symlink_to_notebooks model_storage_dir repo_storage_dir activate_xformers activate_deepdanbooru pip_cache_dir install_pip_xformers
    test = [symlink_to_notebooks, model_storage_dir, repo_storage_dir, activate_xformers, activate_deepdanbooru, pip_cache_dir, install_pip_xformers]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

from pathlib import Path
import os

%cd "{Path(repo_storage_dir, 'stable-diffusion-webui')}"

!pip install --upgrade pip
!pip install --upgrade wheel setuptools

if pip_cache_dir:
    !pip install git+https://github.com/pixelb/crudini.git
    !mkdir -p "{pip_cache_dir}"
    !python3 -m crudini --set /etc/pip.conf global cache-dir "{pip_cache_dir}"
    !echo "Set pip cache directory: $(pip cache dir)"

# Uninstall PyTorch and some other libraries so the WebUI can install the versions it needs
!pip uninstall -y torch torchvision torchaudio protobuf

# Import launch.py which will automatically run the install script but not launch the WebUI.
import launch
launch.prepare_environment()

# Install things for this notebook
!pip install requests gdown bs4 markdownify

# The installer isn't installing deepdanbooru right now so we'll do it manually.
if activate_deepdanbooru:
    # https://github.com/KichangKim/DeepDanbooru/releases
    !pip install "git+https://github.com/KichangKim/DeepDanbooru.git@v3-20211112-sgd-e28#egg=deepdanbooru[tensorflow]" # $(curl --silent "https://api.github.com/KichangKim/DeepDanbooru/releases/latest" | grep '"tag_name":' | sed -E 's/.*"([^"]+)".*/\1/')#egg=deepdanbooru[tensorflow]" # tensorflow==2.10.0 tensorflow-io==0.27.0 flatbuffers==1.12

# We need to install xformers first so that the WebUI installer can install the correct version of PyTorch afterwards
if activate_xformers:
    if install_pip_xformers:
        print('Installing xformers through pip...')
        !pip install --no-dependencies xformers
    else:
        import subprocess
        from glob import glob
        def download_release(url, binary_name='xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl'):
            tmp_dir = subprocess.check_output(['mktemp', '-d']).decode('ascii').strip('\n')
            !wget "{url}" -O "{tmp_dir}/{binary_name}"
            return os.path.join(tmp_dir, binary_name)

        xformers_whl = None
        found_xformers_whls = glob('/notebooks/xformers-*')
        if len(found_xformers_whls) == 1:
            print('Installing xformers using your pre-built wheel...')
            xformers_whl = found_xformers_whls[0]
            delete_whl = False
        elif len(found_xformers_whls) > 1:
            print('Found more than one Xformers wheel in /notebooks so not doing anything!')
        else:
            print('Installing xformers from wheels on Github...')
            delete_whl = True
            # Set up pip packages
            # !pip uninstall -y torch  torchvision torchaudio # Remove existing pytorch install.
            # !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113 # Install pytorch for cuda 11.3
            s = subprocess.getoutput('nvidia-smi')
            if 'A4000' in s:
                xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/raw/main/a4000/xformers-0.0.18%2Bda27862.d20230413-cp39-cp39-linux_x86_64.whl')
            elif 'A5000' in s:
                xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A5000-Nov-1-2022/a5000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
            elif 'A6000' in s:
                xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A6000-Nov-1-2022/a6000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
            elif 'P5000' in s:
                xformers_whl = download_release('https://raw.githubusercontent.com/Cyberes/xformers-compiled/main/p5000/xformers-0.0.16%2B6f3c20f.d20230127-cp39-cp39-linux_x86_64.whl')
            elif 'RTX 4000' in s:
                xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/RTX-4000-Nov-1-2022/rtx4000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
            elif 'RTX 5000' in s:
                xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/RTX-5000-Nov-1-2022/rtx5000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
            elif 'A100' in s:
                xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A100-Nov-1-2022/a100-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
            elif 'M4000' in s:
                print('xformers for M4000 hasn\'t been built yet.')
                # xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A100-Nov-1-2022/a100-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
            else:
                print('GPU not matched to xformers binary so a one-size-fits-all binary was installed. If you have any issues, please build xformers using the Tools block below.')
                xformers_whl = download_release('https://raw.githubusercontent.com/Cyberes/xformers-compiled/main/various/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl')
        if xformers_whl:
            !pip uninstall -y xformers
            # We're going to install xformers without installing any of its dependencies since they should already be installed.
            # If you have any issues then replacing --no-dependencies with --force-reinstall
            !pip install --no-dependencies "{xformers_whl}"
            if delete_whl:
                !rm -rf "{xformers_whl}"
# Make sure important directories exists
!mkdir -p "{model_storage_dir}/hypernetworks"
!mkdir -p "{model_storage_dir}/vae"
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/models/hypernetworks"
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/models/VAE"
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/models/Lora"
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/log/images"

!echo -e "\n===================================\nDone! If you're seeing this the process has exited successfully.\n"
#######################
#######################
#######################
#######################    REQUIRED FIXUPS
#######################
#######################
#######################


!pip install --upgrade "tensorflow"
!pip install "tensorboardX"
!pip install --upgrade "xformers==0.0.20" "protobuf==3.20.3"
!pip install --upgrade "pydantic<2>1.12" "spacy>3.4<4" "fastapi-class"
#######################
#######################
#######################
#######################
#######################
#######################
#######################


try:
    %store -r model_storage_dir repo_storage_dir link_novelai_anime_vae search_paperspace_datasets
    test = [model_storage_dir, repo_storage_dir, link_novelai_anime_vae, search_paperspace_datasets]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

import os
from glob import glob
from pathlib import Path
import sys

model_storage_dir = Path(model_storage_dir)

if not model_storage_dir.exists():
    print('Your model storage directory does not exist:', model_storage_dir)
    sys.exit(1)

webui_root_model_path = Path(repo_storage_dir, 'stable-diffusion-webui/models')
webui_sd_model_path = Path(webui_root_model_path, 'Stable-diffusion')
webui_hypernetwork_path = Path(webui_root_model_path, 'hypernetworks')
webui_vae_path = Path(webui_root_model_path, 'VAE')
webui_lora_model_path = Path(webui_root_model_path, 'Lora')

def delete_broken_symlinks(dir):
    deleted = False
    dir = Path(dir)
    for file in dir.iterdir():
        if file.is_symlink() and not file.exists():
            print('Symlink broken, removing:', file)
            file.unlink()
            deleted = True
    if deleted:
        print('')

def create_symlink(source, dest):
    if os.path.isdir(dest):
        dest = Path(dest, os.path.basename(source))
    if not dest.exists():
        os.symlink(source, dest)
    print(source, '->', Path(dest).absolute())

# Check for broken symlinks and remove them
print('Removing broken symlinks...')
delete_broken_symlinks(webui_sd_model_path)
delete_broken_symlinks(webui_hypernetwork_path)
delete_broken_symlinks(webui_vae_path)
delete_broken_symlinks(webui_lora_model_path)

def link_ckpts(source_path):
    # Link .ckpt and .safetensor/.st files (recursive)
    print('\nLinking .ckpt and .safetensor/.safetensors/.st files in', source_path)
    source_path = Path(source_path)
    for file in [p for p in source_path.rglob('*') if p.suffix in ['.ckpt', '.safetensor', '.safetensors', '.st']]:
        if Path(file).parent.parts[-1] not in ['hypernetworks', 'vae'] :
            if not (webui_sd_model_path / file.name):
                print('New model:', file.name)
            create_symlink(file, webui_sd_model_path)
    # Link config yaml files
    print('\nLinking config .yaml files in', source_path)
    for file in model_storage_dir.glob('*.yaml'):
        create_symlink(file, webui_sd_model_path)


link_ckpts(model_storage_dir)

# Link hypernetworks
print('\nLinking hypernetworks...')
hypernetwork_source_path = Path(model_storage_dir, 'hypernetworks')
if hypernetwork_source_path.is_dir():
    for file in hypernetwork_source_path.iterdir():
        create_symlink(hypernetwork_source_path / file, webui_hypernetwork_path)
else:
    print('Hypernetwork storage directory not found:', hypernetwork_source_path)

# Link VAEs
print('\nLinking VAEs...')
vae_source_path = Path(model_storage_dir, 'vae')
if vae_source_path.is_dir():
    for file in vae_source_path.iterdir():
        create_symlink(vae_source_path / file, webui_vae_path)
else:
    print('VAE storage directory not found:', vae_source_path)

# Link Lora
print('\nLinking Loras...')
lora_source_path = Path(model_storage_dir, 'Lora')
if lora_source_path.is_dir():
    for file in lora_source_path.iterdir():
        create_symlink(lora_source_path / file, webui_lora_model_path)
else:
    print('Lora storage directory not found:', lora_source_path)

# Link the NovelAI files for each of the NovelAI models
print('\nLinking NovelAI files for each of the NovelAI models...')
for model in model_storage_dir.glob('novelai-*.ckpt'):
    yaml = model.stem + '.yaml'
    if os.path.exists(yaml):
        print('New NovelAI model config:', yaml)
        create_symlink(yaml, webui_sd_model_path)

if link_novelai_anime_vae:
    print('\nLinking NovelAI anime VAE...')
    for model in model_storage_dir.glob('novelai-*.ckpt'):
        if (model_storage_dir / 'hypernetworks' / 'animevae.pt').is_file():
            vae = model.stem + '.vae.pt'
            if not os.path.exists(webui_vae_path):
                print(f'Linking NovelAI {vae} and {model}')
            create_symlink(model_storage_dir / 'hypernetworks' / 'animevae.pt', webui_vae_path)
        else:
            print(f'{model_storage_dir}/hypernetworks/animevae.pt not found!')

if search_paperspace_datasets:
    if Path('/datasets').is_dir():
        link_ckpts('/datasets')
    else:
        print('\nNo datasets mounted!')
        
!sudo apt-get install -y libxext-dev libx11-dev x11proto-gl-dev libtool xvfb libtool-bin libosmesa6-dev libgles2-mesa-dev libegl1-mesa-dev
!pip install pegl pyrender
%cd /notebooks/libglvnd
!make install
!ldconfig -N -v | grep libEGL
!python -c 'import pyrender; from pyrender.platforms import egl; from OpenGL.EGL import *; devices = egl.query_devices(); print(devices)'


# SECOND CELL BACKGROUND AN X FRAMEBUFFER

re-run only if X crashes

Without an active FB vispy throws a shit fit

Mesa EGL is used to render

TODO Force NVIDIA EGL binding somehow...

In [16]:
from subprocess import Popen, PIPE
import shlex
p = Popen(shlex.split('/usr/bin/Xvfb :0 -screen 0 1024x768x24 -ac +extension GLX +render -noreset'), bufsize=1, universal_newlines=True)

# FINAL CELL STARTS WEBUI

RE-RUN ONLY THIS CELL TO RESTART WEBUI IF NEEDED


In [ ]:
!export DISPLAY=:0
!export MESA_GL_VERSION_OVERRIDE=3.3

import os

os.environ['DISPLAY']= ':0'
os.environ['MESA_GL_VERSION_OVERRIDE']= '3.3'

try:
    %store -r model_storage_dir repo_storage_dir activate_xformers activate_deepdanbooru activate_medvram disable_pickle_check gradio_port gradio_auth ui_theme insecure_extension_access gradio_queue
    test = [model_storage_dir, repo_storage_dir, activate_xformers, activate_deepdanbooru, activate_medvram, disable_pickle_check, gradio_port, gradio_auth, ui_theme, insecure_extension_access, gradio_queue]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

from pathlib import Path
%cd "{Path(repo_storage_dir, 'stable-diffusion-webui')}"
insecure_extension_access = True
# Code to set the options you want as defined in the very first block
x_arg = '--xformers' if activate_xformers else ''
dd_arg = '--deepdanbooru' if activate_deepdanbooru else ''
mvram_arg = '--medvram' if activate_medvram else ''
pickled = '--disable-safe-unpickle' if disable_pickle_check else ''
port = f'--port {gradio_port}' if gradio_port else '--share'
auth = f'--gradio-auth {gradio_auth} --enable-insecure-extension-access' if gradio_auth else ''
theme = f'--theme {ui_theme}' if ui_theme else ''
insecure_extension_access = '--enable-insecure-extension-access' if insecure_extension_access else ''
queue = '--gradio-queue' if gradio_queue else ''

# Launch args go below:
!python -c "import vispy; print(vispy.sys_info())"
!python webui.py {x_arg} {dd_arg} {mvram_arg} {pickled} {port} {auth} {theme} {queue} {insecure_extension_access}